In [1]:
import pandas as pd
from numpy.random import default_rng
from time import time
from mtjjv_sketch import *
from stainer_new import *
from history import *

In [2]:
df = pd.DataFrame([(0, 'Cat'), (1, 'Dog'), (2, 'Rabbit'), (3, 'Cat'), (4, 'Cat'), (5, 'Dog')],
                  columns=('id', 'class'))

In [3]:
stain = ShuffleStainer("Shuffle")

In [4]:
res = DirtyDF(df, seed = 214).add_stainers(stain).add_stainers(stain).run_all_stainers()
res.get_df()

,id,class
0,2,Rabbit
1,5,Dog
2,4,Cat
3,1,Dog
4,0,Cat
5,3,Cat


In [5]:
res.history[0].row_map

{3: 0, 0: 1, 4: 2, 2: 3, 1: 4, 5: 5}

In [6]:
res.history[1].row_map

{3: 0, 5: 1, 2: 2, 4: 3, 1: 4, 0: 5}

In [7]:
res.get_mapping(axis = 0)

{0: 4, 1: 3, 2: 0, 3: 5, 4: 2, 5: 1}

In [8]:
new_ddf = DirtyDF(df, seed = 214).add_stainers(stain).run_stainer()
new_ddf.add_stainers(stain).run_stainer().get_df()

,id,class
0,2,Rabbit
1,5,Dog
2,4,Cat
3,1,Dog
4,0,Cat
5,3,Cat


In [9]:
res.history[0].message

'Shuffling'

In [11]:
res.history[1].message

'Shuffling'